# Emma Source Language

Emma is a *domain-specific language (DSL)* for parallel data analysis embedded in Scala. As such, Emma accepts a subset of Scala as valid source expressions. The language induced by this set is called the *Emma Source* and is introduced by example below.

This notebook is intented for developers who want to work in the Emma compiler.

For an introduction to the Emma DSL by example, please check the `EmmaByExample` notebook.

## Notebook Setup

The snippet assumes that you have installed the current version of the Emma in your local Maven repository before opeining the notebook. If this is not the case, you should do this from the project root with the following Maven command.

```
mvn clean install -DskipTests
```

We can then load the `emma-language` artifact as follows.

In [1]:
import java.nio.file.Paths

// register the maven repository
classpath.addRepository(
  s"file://${System.getenv("HOME")}/.m2/repository/"
)

// add the required manen modules
classpath.add("eu.stratosphere" % "emma-language" % "1.0-SNAPSHOT")

// add the test-classes from emma-language
val testClasses = Paths.get("../emma-language/target/test-classes").toAbsolutePath().normalize().toString()
classpath.addPath(testClasses)

Adding 19 artifact(s)


import java.nio.file.Paths
testClasses: String = "/home/alexander/workspace/java/projects/emma/emma-language/target/test-classes"

## Compiler Infrastructure

Scala offers facilities for both compile-time and runtime reflection (for more information, read the [Reflection Overview](http://docs.scala-lang.org/overviews/reflection/overview.html) documentation).
While the two APIs are mostly similar, there are some subtle differences that need to be considered.

Emma unifies the two approaches under a single `Compiler` interface with two implementations:

- [`MacroCompiler`](https://github.com/stratosphere/emma/blob/newir/emma-language/src/main/scala/eu/stratosphere/emma/compiler/MacroCompiler.scala) (which operates at compile-time), and 
- [`RuntimeCompiler`](https://github.com/stratosphere/emma/blob/newir/emma-language/src/main/scala/eu/stratosphere/emma/compiler/RuntimeCompiler.scala) (which operates at runtime).

This unified approach gives compiler developers the freedom to decide ad-hoc which parts of the compilation pipeline have to be performed statically and which dynamically.

The examples below are illustrated based on a `RuntimeCompiler` instance that can be created as follows.

In [2]:
import eu.stratosphere.emma.compiler.RuntimeCompiler
val compiler = new RuntimeCompiler()

import eu.stratosphere.emma.compiler.RuntimeCompiler
compiler: eu.stratosphere.emma.compiler.RuntimeCompiler = eu.stratosphere.emma.compiler.RuntimeCompiler@34750a3a

Once we have a `Compiler` instance, we can import the (path-dependent) Scala reflection universe.

In [3]:
import compiler.universe._

import compiler.universe._

## Compiler Pipelines

`Compiler` mixes in a number of traits which introduce compiler functionality in a modular way.

Each trait defines a set of stateless transformations that consume a Scala `Tree` and produce a new `Tree`.

Compilation pipelines are defined in the so-called *point-free* style by means of chaining such transformation functions using the `andThen` combinator.

For example, a trivial compiler pipeline that just typechecks a reified Scala code snippet can be defined as follows.

In [4]:
def typeCheck[T]: Expr[T] => Tree = {
  (_: Expr[T]).tree
} andThen {
  compiler.Type.check(_: Tree)
}

defined function typeCheck

To see this pipeline in action, let us reify some code snippets and pass them as arguments to the `typeCheck` function.

In [5]:
val QandA = typeCheck(reify {
  val Q = "What is the meaning of Life, the Universe, and Everything?"
  val A = 42
  (Q, A)
})

val QuestionAndAnswer = typeCheck(reify {
  val question = "What is the meaning of Life, the Universe, and Everything?"
  val answer = 42
  (question, answer)
})

QandA: Tree = {
  val Q: String = "What is the meaning of Life, the Universe, and Everything?";
  val A: Int = 42;
  Tuple2.apply[String, Int](Q, A)
}
QuestionAndAnswer: Tree = {
  val question: String = "What is the meaning of Life, the Universe, and Everything?";
  val answer: Int = 42;
  Tuple2.apply[String, Int](question, answer)
}

Trees can be converted to source code with the `compiler.asSource` method.

In [6]:
def asSource(name: String)(tree: Tree): Unit = {
  import compiler.{asSource}
  // print the returned source in a Markdwon block
  display.markdown(
    s"""
     |```scala
     |${compiler.asSource(name)(tree)}
     |```
     |""".stripMargin)
}

asSource("QandA")(QandA)
asSource("QuestionAndAnswer")(QuestionAndAnswer)


```scala
QandA
--------------------------------------------------------------------------------
{
  val Q = "What is the meaning of Life, the Universe, and Everything?";
  val A = 42;
  Tuple2.apply[String, Int](Q, A)
}
--------------------------------------------------------------------------------

```



```scala
QuestionAndAnswer
--------------------------------------------------------------------------------
{
  val question = "What is the meaning of Life, the Universe, and Everything?";
  val answer = 42;
  Tuple2.apply[String, Int](question, answer)
}
--------------------------------------------------------------------------------

```


defined function asSource

The compiler also offers a function `alphaEq` that can be used to check whether two Scala trees are [alpha equivalent](https://en.wikipedia.org/wiki/Lambda_calculus#Alpha_equivalence).

In [7]:
def alphaEq(lhs: Tree, rhs: Tree): Boolean = {
  import compiler.{alphaEq, Eq, Neq}
  import org.scalactic.{Bad, Good, Or}
  // returns a Scalactic `Eq Or Neq` value where
  //   type Eq = Unit
  //   case class Neq(lhs: Tree, rhs: Tree, msg: String)
  compiler.alphaEq(lhs, rhs) match { 
    case Good(_: Eq) => true
    case Bad(_: Neq) => false
  }
}

alphaEq(QandA, QuestionAndAnswer)

defined function alphaEq
res6_1: Boolean = true

## Example Data

For the examples below, we first define some test data.

In [8]:
import eu.stratosphere.emma.testschema.Literature.{Book,Character}

val hhBook = Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams")

val arthur = Character("Arthur Dent", hhBook)
val zaphod = Character("Zaphod Beeblebrox", hhBook)
val jeltz  = Character("Prostetnic Vogon Jeltz", hhBook)

import eu.stratosphere.emma.testschema.Literature.{Book,Character}
hhBook: eu.stratosphere.emma.testschema.Literature.Book = Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams")
arthur: eu.stratosphere.emma.testschema.Literature.Character = Character(
  "Arthur Dent",
  Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams")
)
zaphod: eu.stratosphere.emma.testschema.Literature.Character = Character(
  "Zaphod Beeblebrox",
  Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams")
)
jeltz: eu.stratosphere.emma.testschema.Literature.Character = Character(
  "Prostetnic Vogon Jeltz",
  Book("The Hitchhiker's Guide to the Galaxy", "Douglas Adams")
)

## Language Constructs

The [`Source`](https://github.com/stratosphere/emma/blob/newir/emma-language/src/main/scala/eu/stratosphere/emma/compiler/lang/Source.scala) trait defines a `Source` object that contains facilities for handling Emma Source expressions.

In [9]:
import compiler.Source

import compiler.Source

`Source.Language` contains objects that represent the primitive constructs that form the Emma Source language (i.e., the set of symbos in the Emma Source grammar).

In [10]:
import Source.Language._

import Source.Language._

The obvious way to represent Emma Language programs is directly, i.e. through trees over the primitive constructs defined in `Source.Language`.

Instead of taking this route, we reuse Scala ASTs as an underlying program representation model. This allows us to reuse much of the infrastructure already provided by Scala's reflection API and keep the Emma codebase small.

Each `Source.Language` object serves as a bidirectional mapping between the Emma Source construct and its corresponding Scala AST representation.

As such, it offers means to

- extract instances of the modeled construct from a Scala `Tree` (using the `unapply` method), as well as to 
- create a Scala `Tree` that represents an instance of the modeled construct (using the `apply` method).

In the sections below we provide an overview of the supported constructs (as quoted Scala source code), along with small construction and pattern matching examples.

### Atomic Expressions

Atomic expressions are either 

- literals (modeled by `lit`), or 
- references (modeled by `ref`).

#### Literals

In [11]:
// code quotation
val code: Tree = typeCheck(reify {
  "What is the meaning of Life, the Universe, and Everything?"
  42
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@lit(_) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case lit(v) => lit(v)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  "What is the meaning of Life, the Universe, and Everything?";
  42
}
act: Seq[Tree] = List("What is the meaning of Life, the Universe, and Everything?", 42)
exp: Seq[Tree] = List("What is the meaning of Life, the Universe, and Everything?", 42)
res10_3: Boolean = true

#### References

In [12]:
// code quotation
val code: Tree = typeCheck(reify {
  arthur
  zaphod
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@ref(_) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case ref(sym) => ref(sym)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  cmd11.$ref$cmd7.arthur;
  cmd11.$ref$cmd7.zaphod
}
act: Seq[Tree] = List(cmd11, cmd11)
exp: Seq[Tree] = List(cmd11, cmd11)
res11_3: Boolean = true

### Terms

Valid terms are all literals, as well as

- selections (modeled by `sel`),
- applications (modeled by `app`),
- class instantiations (modeled by `inst`),
- lambda terms (modeled by `lambda`),
- type ascriptions (modeled by `typed`).

Terms can be constructed from other terms as long as the composition type-safe. Since the Emma compiler operates on typechecked Scala ASTs, we can always assume (as a precondition) that this is always the case. To keep this constraint invariant, tranformations implemented in the Emma compiler must be sound with respect to the typing rules.

#### Selections

In [13]:
// code
val code = typeCheck(reify {
  arthur.name
  zaphod.name
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@sel(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case sel(tgt, sym) => sel(tgt, sym)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  cmd12.$ref$cmd7.arthur.name;
  cmd12.$ref$cmd7.zaphod.name
}
act: Seq[Tree] = List(
  cmd12.$ref$cmd7.arthur.name,
  cmd12.$ref$cmd7.arthur,
  cmd12.$ref$cmd7,
  cmd12.$ref$cmd7.zaphod.name,
  cmd12.$ref$cmd7.zaphod,
  cmd12.$ref$cmd7
)
exp: Seq[Tree] = List(
  cmd12.$ref$cmd7.arthur.name,
  cmd12.$ref$cmd7.arthur,
  cmd12.$ref$cmd7,
  cmd12.$ref$cmd7.zaphod.name,
  cmd12.$ref$cmd7.zaphod,
  cmd12.$ref$cmd7
)
res12_3: Boolean = true

#### Applications

In [14]:
// code
val code = typeCheck(reify {
  Character("Trillian", hhBook)
  Character("Marvin", hhBook)
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@app(_, _, _@_*) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case app(fn, targs, argss@_*) => app(fn, targs: _*)(argss: _*)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  Literature.Character.apply("Trillian", cmd13.$ref$cmd7.hhBook);
  Literature.Character.apply("Marvin", cmd13.$ref$cmd7.hhBook)
}
act: Seq[Tree] = List(
  Literature.Character.apply("Trillian", cmd13.$ref$cmd7.hhBook),
  Literature.Character.apply("Marvin", cmd13.$ref$cmd7.hhBook)
)
exp: Seq[Tree] = List(
  Literature.Character.apply("Trillian", cmd13.$ref$cmd7.hhBook),
  Literature.Character.apply("Marvin", cmd13.$ref$cmd7.hhBook)
)
res13_3: Boolean = true

#### Class Instantiations

In [15]:
// code
val code = typeCheck(reify {
  new Character("Trillian", hhBook)
  new Character("Marvin", hhBook)
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@inst(_, _, _@_*) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case inst(clazz, targs, argss@_*) => inst(clazz, targs: _*)(argss: _*)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  new Literature.Character("Trillian", cmd14.$ref$cmd7.hhBook);
  new Literature.Character("Marvin", cmd14.$ref$cmd7.hhBook)
}
act: Seq[Tree] = List(
  new Literature.Character("Trillian", cmd14.$ref$cmd7.hhBook),
  new Literature.Character("Marvin", cmd14.$ref$cmd7.hhBook)
)
exp: Seq[Tree] = List(
  new eu.stratosphere.emma.testschema.Literature.Character("Trillian", cmd14.$ref$cmd7.hhBook),
  new eu.stratosphere.emma.testschema.Literature.Character("Marvin", cmd14.$ref$cmd7.hhBook)
)
res14_3: Boolean = true

#### Lambdas

In [16]:
// code
val code = typeCheck(reify {
  (x: Character) => x.name
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@lambda(_, _, _) => tree
} map { tree =>
  compiler.Owner.at(compiler.Owner.enclosing)(tree)
}

// tree construction
val exp: Seq[Tree] = act map {
  case lambda(sym, params, body) => lambda(params: _*)(body)
} map { tree =>
  compiler.Owner.at(compiler.Owner.enclosing)(tree)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = ((x: eu.stratosphere.emma.testschema.Literature.Character) => x.name)
act: Seq[Tree] = List(((x: eu.stratosphere.emma.testschema.Literature.Character) => x.name))
exp: Seq[Tree] = List(((x: eu.stratosphere.emma.testschema.Literature.Character) => x.name))
res15_3: Boolean = true

#### Type-Ascribed Trees

In [17]:
// code
val code = typeCheck(reify {
  (zaphod: Character)
  (jeltz: Character)
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@typed(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case typed(tree, tpe) => typed(tree, tpe)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  (cmd16.$ref$cmd7.zaphod: eu.stratosphere.emma.testschema.Literature.Character);
  (cmd16.$ref$cmd7.jeltz: eu.stratosphere.emma.testschema.Literature.Character)
}
act: Seq[Tree] = List(
  (cmd16.$ref$cmd7.zaphod: eu.stratosphere.emma.testschema.Literature.Character),
  (cmd16.$ref$cmd7.jeltz: eu.stratosphere.emma.testschema.Literature.Character)
)
exp: Seq[Tree] = List(
  (cmd16.$ref$cmd7.zaphod: eu.stratosphere.emma.testschema.Literature.Character),
  (cmd16.$ref$cmd7.jeltz: eu.stratosphere.emma.testschema.Literature.Character)
)
res16_3: Boolean = true

### State

Terms can be given a name that can be later referenced in a value definition (modeled by `val_`). 

Mutable names are allowed at this level and can be reassigned to a new value (modeled by `assign`).

Sequential composition of value definitions and terms yields a block (modeled by `block`), which decomposes into a sequence of statements (`stats`) and a final return expression (`expr`).

In [18]:
// code
val code = typeCheck(reify {
  // block statements
  val trillian = new Character("Trillian", hhBook)   // immutable value definition
  var marvin   = Character("Marwin", hhBook)         // mutable value definition
  marvin       = new Character("Marwin", hhBook)     // mutable value assignment
  // block return expression
  s"${trillian.name} and ${marvin.name} are friends" // a composite term
})

code: Tree = {
  val trillian: eu.stratosphere.emma.testschema.Literature.Character = new Literature.Character("Trillian", cmd17.$ref$cmd7.hhBook);
  var marvin: eu.stratosphere.emma.testschema.Literature.Character = Literature.Character.apply("Marwin", cmd17.$ref$cmd7.hhBook);
  marvin = new Literature.Character("Marwin", cmd17.$ref$cmd7.hhBook);
  StringContext.apply("", " and ", " are friends").s(trillian.name, marvin.name)
}

#### Value Definitions

In [19]:
// tree matching
val act: Seq[Tree] = code collect {
  case tree@val_(_, _, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case val_(lhs, rhs, flags) => val_(lhs, rhs, flags)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

act: Seq[Tree] = List(
  val trillian: eu.stratosphere.emma.testschema.Literature.Character = new Literature.Character("Trillian", cmd17.$ref$cmd7.hhBook),
  var marvin: eu.stratosphere.emma.testschema.Literature.Character = Literature.Character.apply("Marwin", cmd17.$ref$cmd7.hhBook)
)
exp: Seq[Tree] = List(
  val trillian: eu.stratosphere.emma.testschema.Literature.Character = new Literature.Character("Trillian", cmd17.$ref$cmd7.hhBook),
  var marvin: eu.stratosphere.emma.testschema.Literature.Character = Literature.Character.apply("Marwin", cmd17.$ref$cmd7.hhBook)
)
res18_2: Boolean = true

#### Assignments

In [20]:
// tree matching
val act: Seq[Tree] = code collect {
  case tree@assign(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case assign(lhs, rhs) => assign(lhs, rhs)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

act: Seq[Tree] = List(marvin = new Literature.Character("Marwin", cmd17.$ref$cmd7.hhBook))
exp: Seq[Tree] = List(marvin = new Literature.Character("Marwin", cmd17.$ref$cmd7.hhBook))
res19_2: Boolean = true

#### Blocks

In [21]:
// tree matching
val act: Seq[Tree] = code collect {
  case tree@block(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case block(stats, expr) => block(stats, expr)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

act: Seq[Tree] = List(
  {
  val trillian: eu.stratosphere.emma.testschema.Literature.Character = new Literature.Character("Trillian", cmd17.$ref$cmd7.hhBook);
  var marvin: eu.stratosphere.emma.testschema.Literature.Character = Literature.Character.apply("Marwin", cmd17.$ref$cmd7.hhBook);
  marvin = new Literature.Character("Marwin", cmd17.$ref$cmd7.hhBook);
  StringContext.apply("", " and ", " are friends").s(trillian.name, marvin.name)
}
)
exp: Seq[Tree] = List(
  {
  val trillian: eu.stratosphere.emma.testschema.Literature.Character = new Literature.Character("Trillian", cmd17.$ref$cmd7.hhBook);
  var marvin: eu.stratosphere.emma.testschema.Literature.Character = Literature.Character.apply("Marwin", cmd17.$ref$cmd7.hhBook);
  marvin = new Literature.Character("Marwin", cmd17.$ref$cmd7.hhBook);
  StringContext.apply("", " and ", " are friends").s(trillian.name, marvin.name)
}
)
res20_2: Boolean = true

### Control Flow

Control flow is supported through the following constructs: 

- if-then-else statements (modeled by `branch`),
- while-do loops (modeled by `dowhile`), and
- while-do loops (modeled by `whiledo`).

#### If-Then-Else Expressions

In [22]:
// code
val code = typeCheck(reify {
  if (zaphod.name.startsWith("Zaphod")) 
    zaphod.name
  else
    "Zaphod"
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@branch(_, _, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case branch(cond, thn, els) => branch(cond, thn, els)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = if (cmd21.$ref$cmd7.zaphod.name.startsWith("Zaphod"))
  cmd21.$ref$cmd7.zaphod.name
else
  "Zaphod"
act: Seq[Tree] = List(
  if (cmd21.$ref$cmd7.zaphod.name.startsWith("Zaphod"))
  cmd21.$ref$cmd7.zaphod.name
else
  "Zaphod"
)
exp: Seq[Tree] = List(
  if (cmd21.$ref$cmd7.zaphod.name.startsWith("Zaphod"))
  cmd21.$ref$cmd7.zaphod.name
else
  "Zaphod"
)
res21_3: Boolean = true

#### While-Do Loops

In [23]:
// code
val code = typeCheck(reify {
  var name = zaphod.name
  while (!name.isEmpty) {
    name = name.tail
  }
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@whiledo(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case whiledo(cond, body) => whiledo(cond, body)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  var name: String = cmd22.$ref$cmd7.zaphod.name;
  while$1(){
    if (name.isEmpty().unary_!)
      {
        name = Predef.augmentString(name).tail;
        while$1()
      }
    else
      ()
  }
}
act: Seq[Tree] = List(
  while$1(){
  if (name.isEmpty().unary_!)
    {
      name = Predef.augmentString(name).tail;
      while$1()
    }
  else
    ()
}
)
exp: Seq[Tree] = List(
  while$1(){
  if (name.isEmpty().unary_!)
    {
      name = Predef.augmentString(name).tail;
      while$1()
    }
  else
    ()
}
)
res22_3: Boolean = true

#### Do While Loops

In [24]:
// code
val code = typeCheck(reify {
  var name = zaphod.name
  do {
    name = name.tail
  } while (!name.isEmpty)
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@dowhile(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case dowhile(cond, body) => dowhile(cond, body)
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = {
  var name: String = cmd23.$ref$cmd7.zaphod.name;
  doWhile$1(){
    name = Predef.augmentString(name).tail;
    if (name.isEmpty().unary_!)
      doWhile$1()
    else
      ()
  }
}
act: Seq[Tree] = List(
  doWhile$1(){
  name = Predef.augmentString(name).tail;
  if (name.isEmpty().unary_!)
    doWhile$1()
  else
    ()
}
)
exp: Seq[Tree] = List(
  doWhile$1(){
  name = Predef.augmentString(name).tail;
  if (name.isEmpty().unary_!)
    doWhile$1()
  else
    ()
}
)
res23_3: Boolean = true

### Pattern Matching

Finally, Emma Source also models pattern matching through the `match_`, `case_`, and `bind` constructs.

In [25]:
// code
val code = typeCheck(reify {
  zaphod match {
    case char@Character("Zaphod Beeblebrox", _) => char.book
  }
})

// tree matching
val act: Seq[Tree] = code collect {
  case tree@match_(_, _) => tree
}

// tree construction
val exp: Seq[Tree] = act map {
  case match_(selector, cases) => match_(selector, cases map {
    case cs@case_(pat, guard, body) => case_(pat, guard, body)
  })
}

// alpha-equivalence check
(act zip exp) forall ((alphaEq _).tupled)

code: Tree = cmd24.$ref$cmd7.zaphod match {
  case (char @ (name: String, book: eu.stratosphere.emma.testschema.Literature.Book)eu.stratosphere.emma.testschema.Literature.Character("Zaphod Beeblebrox", _)) => char.book
}
act: Seq[Tree] = List(
  cmd24.$ref$cmd7.zaphod match {
  case (char @ (name: String, book: eu.stratosphere.emma.testschema.Literature.Book)eu.stratosphere.emma.testschema.Literature.Character("Zaphod Beeblebrox", _)) => char.book
}
)
exp: Seq[Tree] = List(
  cmd24.$ref$cmd7.zaphod match {
  case (char @ (name: String, book: eu.stratosphere.emma.testschema.Literature.Book)eu.stratosphere.emma.testschema.Literature.Character("Zaphod Beeblebrox", _)) => char.book
}
)
res24_3: Boolean = true